In [182]:
from lxml import html
import json
import pprint

In [178]:
!ls

carshop  carshop_analysis.ipynb  carshopper.json  scrapy.cfg


In [179]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [211]:
with open('carshopper.json', 'r') as myfile:
    data=myfile.read()
    
obj = json.loads(data)
# pages_html = obj[0]['Response']
# pages_html
pprint.pprint(obj[0])
len(obj)

{'1787-1153928_2': {'body': 'Suv',
                    'date': '2020-01-09',
                    'deriviative': '2.0 Elite Auto',
                    'engine': '2.0',
                    'fuel': 'Petrol',
                    'location': 'gauteng',
                    'make': 'Hyundai',
                    'mileage': '5k',
                    'model': 'Tucson',
                    'price': 'R 664,900',
                    'transmision': 'Automatic',
                    'url': '/used-cars/hyundai/2019-hyundai-tucson-2-0-elite-auto-gauteng/1787-1153928_2',
                    'year': '2019'},
 '1841-643528': {'body': 'Suv',
                 'date': '2020-01-09',
                 'deriviative': '200 F-Sport',
                 'engine': '2.0',
                 'fuel': 'Petrol',
                 'location': 'eastern-cape',
                 'make': 'Lexus',
                 'mileage': '2k',
                 'model': 'UX',
                 'price': 'R 669,900',
                 'transmision': 

19

In [212]:
len(obj)

339

In [213]:
from collections import ChainMap
import time 
s = time.time()
# car_info = dict(ChainMap(*obj))

time.time() - s

0.2351057529449463

In [195]:
import pandas as pd


In [202]:
car_df = pd.DataFrame.from_dict(car_info, orient='index').reset_index()

In [209]:
car_df.body.value_counts()

Hatchback        2346
Suv              1787
Sedan             960
Double Cab        502
Single Cab        428
Mpv               109
Ldv                92
Kombi              92
Coupe              64
Convertible        31
Minibus            20
Station Wagon      10
Name: body, dtype: int64

In [191]:
import string

def get_float(input_string):
    ignore_list = string.ascii_letters
    floats = []
    i = 0
    while i < len(input_string):
        temp = ""
        while input_string[i] not in ignore_list:
            temp += input_string[i]
            i+=1
        if temp not in [" ",""]:
            floats.append(temp)
        i += 1
    return floats

phrase = '1.8i 3 door'
get_float(phrase)

['1.8', ' 3 ']

In [147]:
# tree = html.fromstring(response)


provinces = ['western-cape' ,'eastern-cape','northern-Cape', 'north-west', 'free-state', 'kwazulu-natal', 'gauteng', 'limpopo' , 'mpumalanga']
pages_html = obj[0]['Response']
tree = html.fromstring(pages_html)

def filter_ids(val):
    return 'used-cars' in val.split('/')
def get_id(val):
    return val.split('/')[-1]
def get_loc(val):
    search_string = val.split('/')[-2]
    return [loc for loc in provinces if loc in search_string]    
url = tree.xpath('//div[@class = "VehicleViewBox"]/a/@href')

location = list(map(get_loc,url))

ID = list(filter(filter_ids, url)) 
ID = list(map(get_id,ID))

def rm_spaces(val):
    return val.strip()
def rm_junk(val):
    return val.replace('\xa0'," ")
def split_info(val):
    return val.split(" ")
year_make_model = tree.xpath('//p[@class="VehicleTextArchive"]/text()')
year_make_model = list(map(rm_spaces,year_make_model))
year_make_model = list(filter(None, year_make_model)) # fastest
year_make_model = list(map(rm_junk,year_make_model))
year_make_model = list(map(split_info,year_make_model))
year = [value[0]  for value in year_make_model ] 
make = [value[1]  for value in year_make_model ] 
model =[" ".join(value[2:]) for value in year_make_model ]


def divide_chunks(l, n): 
      
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 


deriviative_mileage = tree.xpath('//strong/text()')
deriviative_mileage= list(divide_chunks(deriviative_mileage,2))
deriviative = [value[0] for value in deriviative_mileage]
mileage = [value[1] for value in deriviative_mileage]
deriviative,mileage

price = tree.xpath('//span[@class="DealershipNameLeft"]/text()')

capacity_fuel_body_transmission = tree.xpath('//td/p/text()')
capacity_fuel_body_transmission = list(map(rm_spaces,capacity_fuel_body_transmission))
capacity_fuel_body_transmission = list(divide_chunks(capacity_fuel_body_transmission,5) )
capacity = [value[0]  for value in capacity_fuel_body_transmission ]

fuel = [value[1]  for value in capacity_fuel_body_transmission ] 
body =[value[2] for value in capacity_fuel_body_transmission ]
transmission =[value[3] for value in capacity_fuel_body_transmission ]
# carshop = {index : { 'year':y , 'make':mk , 'model': m , 'deriviative':d,'mileage':ml  , 'price': p ,'engine': c , 'fuel':f , 'body':b,'transmision':trans} 
#            for index,y,mk,m,d,ml,p,c,f,b,trans in zip(ID,year,make,model,deriviative,mileage,price,capacity,fuel,body,transmission)    }

JSONDecodeError: Extra data: line 3 column 2 (char 94310)

In [145]:
from datetime import datetime
carshop = {index : { 'year':y , 'make':mk , 'model': m , 'deriviative':d,'mileage':ml  , 'price': p ,'engine': c , 'fuel':f , 'body':b,'transmision':trans , 'url':u ,'location':loc,'date':datetime.strftime(datetime.today(),'%Y-%m-%d') } 
           for index,y,mk,m,d,ml,p,c,f,b,trans,u,loc in zip(ID,year,make,model,deriviative,mileage,price,capacity,fuel,body,transmission,url,location) }


NameError: name 'url' is not defined

In [143]:

for key in carshop.keys():
    if carshop[key]['engine'] == 'N/A':
        carshop[key]['engine'] = [float(value) for value in carshop[key]['deriviative'].split() if ('.' in value)] 
        carshop[key]['engine'] = carshop[key]['engine'][0] if carshop[key]['engine'] else 'N/A'
        
        

In [144]:
import pprint
pprint.pprint(carshop)

{'1621-06128': {'body': 'Sedan',
                'deriviative': '1.6 GLX Auto',
                'engine': 1.6,
                'fuel': 'Petrol',
                'make': 'Toyota',
                'mileage': '300k',
                'model': 'Corolla',
                'price': 'R 29,990',
                'transmision': 'Automatic',
                'year': '1992'},
 '1662-15279': {'body': 'Hatchback',
                'deriviative': 'Lite 1.4i',
                'engine': '1.4',
                'fuel': 'Petrol',
                'make': 'Opel',
                'mileage': '122k',
                'model': 'Corsa',
                'price': 'R 49,900',
                'transmision': 'Manual',
                'year': '2007'},
 '1864-142': {'body': 'Sedan',
              'deriviative': '528i Sport Auto (E39)',
              'engine': 'N/A',
              'fuel': 'Petrol',
              'make': 'BMW',
              'mileage': '297k',
              'model': '5 Series',
              'price': 'R 49,99

In [176]:
l = ['k']
l[0]

'k'